In [ ]:
#!g1.1

#model 1

model_name = "Tatyana/rubert-base-cased-sentiment-new"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))[0]

#!g1.1


class Cls_head(nn.Module):
    
    def __init__(self, last_layer_dim=1024, hidden_layer_size=512, out_size=1):
        super().__init__()
        self.last_layer_dim = last_layer_dim
        self.hidden_layer_size = hidden_layer_size,
        self.out_size = out_size
        
        self.hidden_layer_1 = nn.Linear(last_layer_dim, hidden_layer_size)
        self.relu1 = nn.ReLU(inplace=True)
        self.bn_1 = nn.BatchNorm1d(hidden_layer_size)
        
        self.hidden_layer_2 = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.relu2 = nn.ReLU(inplace=True)
#         self.bn_2 = nn.BatchNorm1d(hidden_layer_size)
        self.dropout2 = nn.Dropout(p=0.3)
        
        
        self.hidden_layer_3 = nn.Linear(hidden_layer_size, hidden_layer_size // 4)
        self.relu3 = nn.ReLU(inplace=True)
        self.bn_3 = nn.BatchNorm1d(hidden_layer_size // 4)
        
        self.cls = nn.Linear(hidden_layer_size // 4, out_size)
        
        
        
    def forward(self, x):
        
        out = self.bn_1(self.relu1(self.hidden_layer_1(x)))
#         out = self.bn_2(self.relu2(self.hidden_layer_2(out)))
        out = self.dropout2(self.relu2(self.hidden_layer_2(out)))
        out = self.bn_3(self.relu3(self.hidden_layer_3(out)))
        out = self.cls(out)
        
        return out
        

class FBERTAcls(nn.Module):

    def __init__(self, model, last_layer_dim=768, hidden_layer_size=256):
        super().__init__()
        
        
        self.BERT = freeze_layers(model, 5)
        
        self.cls_head_1 = Cls_head(last_layer_dim=last_layer_dim, hidden_layer_size=hidden_layer_size, out_size=3)
        self.cls_head_2 = Cls_head(last_layer_dim=last_layer_dim, hidden_layer_size=hidden_layer_size, out_size=5)
        
        
        

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        bert_output = self.BERT(input_ids=input_ids, attention_mask=attention_mask)
#         hidden_out = self.hidden_layer_1(bert_output.last_hidden_state[:, 0, :])

#         print(bert_output.shape)

        bert_output = bert_output.pooler_output
#         print(bert_output.shape)
#         print(bert_output.sum())
        sent_logits = self.cls_head_1(bert_output) 
        
        cat_1_logits = self.cls_head_2(bert_output)

        return sent_logits, cat_1_logits


In [ ]:
#!g1.1
#!g1.1
def freeze_layers(model, num_to_freeze=0):
    
    for layer in list(model.parameters())[:-num_to_freeze]:
        layer.requires_grad = False
#         print(layer.requires_grad)
    return model
        

In [ ]:
#!g1.1
#model 2

model_name = "cointegrated/rubert-tiny"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

base_model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

newmodel = torch.nn.Sequential(*(list(base_model.children())[:-1]))[0]

class Cls_head(nn.Module):
    
    def __init__(self, last_layer_dim=1024, hidden_layer_size=512, out_size=1):
        super().__init__()
        self.last_layer_dim = last_layer_dim
        self.hidden_layer_size = hidden_layer_size,
        self.out_size = out_size
        
        self.hidden_layer_1 = nn.Linear(last_layer_dim, hidden_layer_size)
        self.relu1 = nn.ReLU(inplace=True)
        self.bn_1 = nn.BatchNorm1d(hidden_layer_size)
        
        self.hidden_layer_2 = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.relu2 = nn.ReLU(inplace=True)
#         self.bn_2 = nn.BatchNorm1d(hidden_layer_size)
        self.dropout2 = nn.Dropout(p=0.3)
        
        
        self.hidden_layer_3 = nn.Linear(hidden_layer_size, hidden_layer_size // 4)
        self.relu3 = nn.ReLU(inplace=True)
        self.bn_3 = nn.BatchNorm1d(hidden_layer_size // 4)
        
        self.cls = nn.Linear(hidden_layer_size // 4, out_size)
        
        
        
    def forward(self, x):
        
        out = self.bn_1(self.relu1(self.hidden_layer_1(x)))
#         out = self.bn_2(self.relu2(self.hidden_layer_2(out)))
        out = self.dropout2(self.relu2(self.hidden_layer_2(out)))
        out = self.bn_3(self.relu3(self.hidden_layer_3(out)))
        out = self.cls(out)
        
        return out
        

class FBERTAcls(nn.Module):

    def __init__(self, model, last_layer_dim=768, hidden_layer_size=256):
        super().__init__()
        
        
        self.BERT = freeze_layers(model, 1)
        
        self.cls_head_1 = Cls_head(last_layer_dim=last_layer_dim, hidden_layer_size=hidden_layer_size, out_size=3)
        self.cls_head_2 = Cls_head(last_layer_dim=last_layer_dim, hidden_layer_size=hidden_layer_size, out_size=5)
        
        
        

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        bert_output = self.BERT(input_ids=input_ids, attention_mask=attention_mask)
#         hidden_out = self.hidden_layer_1(bert_output.last_hidden_state[:, 0, :])

#         print(bert_output.shape)

        bert_output = bert_output.pooler_output
#         print(bert_output.shape)
#         print(bert_output.sum())
        sent_logits = self.cls_head_1(bert_output) 
        
        cat_1_logits = self.cls_head_2(bert_output)

        return sent_logits, cat_1_logits




In [1]:
#!g1.1
# model_3 the same as 2

In [ ]:
#!g1.1
